# Cleaning and Conections


#### 1. Library importation

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import unicodedata
import re
import os
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

df = pd.read_csv("RWventas.csv")

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Ciudad           995449 non-null  object
 1   Fecha            995380 non-null  object
 2   Producto         995395 non-null  object
 3   Tipo_Producto    995374 non-null  object
 4   Cantidad         995470 non-null  object
 5   Precio_Unitario  995545 non-null  object
 6   Tipo_Venta       995477 non-null  object
 7   Tipo_Cliente     995579 non-null  object
 8   Descuento        995497 non-null  object
 9   Costo_Envio      995531 non-null  object
 10  Total            995472 non-null  object
dtypes: object(11)
memory usage: 83.9+ MB
None


1.1 Visialization of corrupted data.

In [2]:
for row in df.columns:
    print(f"'{row}' {df[row].unique()}\n")

'Ciudad' ['Antofagasta' 'Monterrey' 'Valparaíso' 'Sevilla' 'Córdoba'
 'Ciudad de México' 'Houston' 'Mendoza' 'Barcelona' 'Chicago' 'Miami'
 'Pereira' 'Cali' 'Concepción' 'Rosario' 'Madrid' 'Arequipa' 'Lima'
 'Santiago' 'Bucaramanga' 'Tijuana' 'Guadalajara' 'Cusco' 'New York'
 'Buenos Aires' 'Valencia' 'Los Angeles' 'Bogotá' 'Trujillo' nan 'Puebla'
 'Medellín' 'Barranquilla' 'Cartagena' 'Barcelona***' 'Córdoba***'
 'Ciudad de México###' 'Barcelona@@@' 'Madrid###' 'Pereira@@@'
 'Santiago***' '  Barcelona   ' 'Miami###' 'vALPARAÍSO' 'cÓRDOBA'
 '  Miami   ' 'Puebla###' 'Medellín@@@' 'Sevilla***' 'Puebla***'
 '  Valparaíso   ' '  Bogotá   ' 'Bogotá###' '  Ciudad de México   '
 '  Pereira   ' 'cUSCO' 'Cali***' 'Barcelona###' 'pEREIRA' '  Cali   '
 'Buenos Aires***' 'cONCEPCIÓN' 'nEW yORK' '  Trujillo   ' '  Sevilla   '
 '  Antofagasta   ' 'Mendoza***' 'gUADALAJARA' 'Trujillo@@@' 'hOUSTON'
 '  Concepción   ' 'Cusco###' 'Madrid@@@' 'Antofagasta###' 'mEDELLÍN'
 'tIJUANA' '  Guadalajara   ' '  M

In [3]:
mis_columns = df.isna().sum()
mis_total = df.isna().sum().sum()

print(f"Missing data per column{ mis_columns}\nTotal data missing {mis_total}")

Missing data per columnCiudad             4551
Fecha              4620
Producto           4605
Tipo_Producto      4626
Cantidad           4530
Precio_Unitario    4455
Tipo_Venta         4523
Tipo_Cliente       4421
Descuento          4503
Costo_Envio        4469
Total              4528
dtype: int64
Total data missing 49831


#### 2. Cleaning DataSet

In [4]:
def clean_city(texto):
    if pd.isna(texto):
        return texto
    
    # text to lower case
    texto = texto.lower().strip()

    # delet ascents
    texto = ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')

    # delet special characters only keep letters
    texto = re.sub(r'[^a-z\s]', '', texto)

    # remove double spaces "  "
    texto = re.sub(r'\s+', ' ', texto)

    return texto

# Normalizing text
cols_texto = ["Ciudad", "Producto", "Tipo_Producto", "Tipo_Venta", "Tipo_Cliente"]
for col in cols_texto:
    df[col] = df[col].apply(clean_city)

# Conversion to numeric
df["Descuento"] = pd.to_numeric(df["Descuento"], errors="coerce")
df["Precio_Unitario"] = pd.to_numeric(df["Precio_Unitario"], errors="coerce")
df["Cantidad"] = pd.to_numeric(df["Cantidad"], errors="coerce")
df["Costo_Envio"] = pd.to_numeric(df["Costo_Envio"], errors="coerce")
df["Total"] = pd.to_numeric(df["Total"], errors="coerce")

# Fill missing
df["Tipo_Cliente"] = df["Tipo_Cliente"].replace("nan", np.nan)
df["Tipo_Cliente"] = df["Tipo_Cliente"].fillna("desconocido")

df["Descuento"] = df["Descuento"].fillna(0)
df["Precio_Unitario"] = df["Precio_Unitario"].fillna(0)
df["Cantidad"] = df["Cantidad"].fillna(0)
df["Costo_Envio"] = df["Costo_Envio"].fillna(0)
df["Total"] = df["Total"].fillna(0)

# Conversion to date format
df["Fecha"] = pd.to_datetime(df["Fecha"], errors="coerce")


# Recalculating "Total" column
df["Total"] = (df["Cantidad"] * df["Precio_Unitario"]) - (df["Descuento"] * df["Cantidad"] * df["Precio_Unitario"]) + df["Costo_Envio"]

df.columns = df.columns.str.lower()

# 6. Save changes in a new .csv

df.to_csv("clean_sales.csv", index=False)

In [5]:
for row in df.columns:
    print(f"'{row}' {df[row].unique()}\n")

'ciudad' ['antofagasta' 'monterrey' 'valparaiso' 'sevilla' 'cordoba'
 'ciudad de mexico' 'houston' 'mendoza' 'barcelona' 'chicago' 'miami'
 'pereira' 'cali' 'concepcion' 'rosario' 'madrid' 'arequipa' 'lima'
 'santiago' 'bucaramanga' 'tijuana' 'guadalajara' 'cusco' 'new york'
 'buenos aires' 'valencia' 'los angeles' 'bogota' 'trujillo' nan 'puebla'
 'medellin' 'barranquilla' 'cartagena']

'fecha' <DatetimeArray>
['2025-11-28 00:00:00', '2025-11-29 00:00:00', '2025-12-07 00:00:00',
 '2025-12-01 00:00:00', '2025-11-18 00:00:00', '2025-11-20 00:00:00',
 '2025-11-15 00:00:00', '2025-11-30 00:00:00', '2025-11-26 00:00:00',
 '2025-11-09 00:00:00', '2025-11-16 00:00:00', '2025-11-12 00:00:00',
 '2025-11-14 00:00:00', '2025-12-04 00:00:00', '2025-11-22 00:00:00',
 '2025-12-02 00:00:00', '2025-11-24 00:00:00', '2025-11-08 00:00:00',
 '2025-11-11 00:00:00',                 'NaT', '2025-11-19 00:00:00',
 '2025-11-23 00:00:00', '2025-11-13 00:00:00', '2025-11-17 00:00:00',
 '2025-12-05 00:00:00', '

#### 3. 